In [9]:
import numpy as np 
from numpy.lib.stride_tricks import sliding_window_view
import matplotlib.pyplot as plt 
from sympy import symbols, Eq, solve
from scipy.integrate import simpson

other_signals = np.genfromtxt('/Users/Jeremy/Desktop/Example calcium imaging Gad2_cre_GCaMP8/MM_20250219-PUI-GadGCaMP8m_Series003/ROIs/ActiveROI_updated.csv', encoding='utf-8-sig', delimiter = ',', usecols=np.arange(0,3), dtype='f8') #adjust for the number of columns there are, should build code for this  
results = {}

speed_of_imaging = float(input("Enter the speed of imaging (images/second): "))
dt = speed_of_imaging
print(f"Speed of imaging is {dt} images/second") 

for row in other_signals.T: #.T at the end turns columns into rows so that the analysis can analyze them as 'columns'
    n = len(row)
    time_end = n*dt
    t = np.arange(0,time_end,dt)                                             
    num_frames=len(t) 

    fhat_ = np.fft.fft(row,num_frames)     
    PSD_ = fhat_ * np.conj(fhat_)/num_frames    
    freq_ = (1/(dt*num_frames))*np.arange(num_frames)    
    L = np.arange(1,np.floor(num_frames/2),dtype='int') 
    #plt.plot(freq_[L], PSD_[L])

    real_amplitude = np.real(PSD_)
    sqrt_real_amplitude = np.sqrt(real_amplitude)
    mean_amplitude = np.mean(sqrt_real_amplitude)
    #print(mean_amplitude)

    indices = PSD_ > mean_amplitude*1.5
    PSD_c = PSD_ * indices
    fhat_filt_ = indices * fhat_ 
    ffilt_ = np.fft.ifft(fhat_filt_)
    complex_ifft_output_ = np.array(ffilt_)
    ffilt_array_ = np.real(complex_ifft_output_)
    #plt.plot(t, ffilt_array_)
    
    #to check if signal is lower than background 
    '''
    if mean_amplitude < b1_mean_amplitude:  
        print("mean amplitude of calcium trace is lower than background")   
    elif mean_amplitude < b2_mean_amplitude:
        print("mean amplitude of calcium trace is lower than background")
    elif mean_amplitude < b3_mean_amplitude: 
        print("mean amplitude of calcium trace is lower than background")
    else: 
        print("mean amplitude of calcium trace is greater than background") #figure out how to call each individual row (actually column but was inverted by other_signals.T)
    '''  
    
    y = symbols('y')
    x = dt 
    eq = Eq(4/x, y) #window size is based on time constant of 4s
    window_size = solve(eq, y)
    window = window_size[0]
    window_size = round(window)
    window_size = int(window_size)

    windows = sliding_window_view(ffilt_array_, window_shape=window_size)   #can use np.convolve with mode='same' if you want the output to have the same length as the input (by calculating the edges with partial windows)
    rolling_mean = windows.mean(axis=1)

    endpoint = time_end-(window_size-1.00000)*dt

    time = np.arange(0, endpoint, dt) #the last window-1 frames are cut off so we have to adjust the time accordingly

    F0 = np.nanmedian(rolling_mean) 
    dF_over_F = (rolling_mean - F0)/F0*100 

    threshold = dF_over_F

    for i in range(len(threshold)):
        value = threshold[i]

        if value < 20:
            threshold[i] = value * 0 
    
    auc = simpson(threshold, time)
    
    freq_events = 1/time_end 
    print(f"The frequency of events in this ROI is {freq_events}/s") 

    activity = auc/time_end
    print(f"Total activity is {activity} %dF/F")

 


Speed of imaging is 0.522 images/second
The frequency of events in this ROI is 0.005552779165972569/s
Total activity is 2.147849225272772 %dF/F
The frequency of events in this ROI is 0.005552779165972569/s
Total activity is 3.71601761979466 %dF/F
The frequency of events in this ROI is 0.005552779165972569/s
Total activity is 1.7982526172759095 %dF/F


Denoising steps  

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

#load data with numpy  
# use this function if you want to specify a range of columns to load: usecols(np.arange())  
signal = np.genfromtxt('/Users/Jeremy/Desktop/Example calcium imaging Gad2_cre_GCaMP8/MM_20230725_Gad2_Ai162/Gad2-Ai162-008803-25_07_23.csv', encoding='utf-8-sig', delimiter = ',', usecols=np.arange(1,2), dtype='f8')
 
speed_of_imaging = float(input("Enter the speed of imaging (images/second): "))
dt = speed_of_imaging
print(f"Speed of imaging is {dt} images/second")
n = len(signal) #counting the number of images there are in the time series 
print(n)

time_end = n*dt #how long it took to take the time series (in seconds)
t = np.arange(0,time_end,dt) #creates a range that allows floats (for precision), where 0 is the start, time_end is the stop (which isn't included in the range), and dt is the step size 


plt.plot(t,signal)
plt.title('Raw')
plt.xlabel("Time (s)")
plt.ylabel("Intensity")

In [ ]:
#FFT with numpy 

fhat = np.fft.fft(signal,n)     #compute the FFT
PSD = fhat * np.conj(fhat)/n    #Power spectrum density (gives you the magnitude of each fourier coefficient and we'll get a vector of powers at each frequency)
freq = (1/(dt*n))*np.arange(n)    #create an x-axis of frequencies 
L = np.arange(1,np.floor(n/2),dtype='int') #only plot first half (real signals are symmetric)

#plotting raw signal and power spectrum density acquired from FFT
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,8))
x1=t
x2=freq[L]
y1=signal
y2=PSD[L]

ax1.plot(x1, y1)
ax1.set_title("Raw")
ax1.set_xlabel('Time(s)')
ax1.set_ylabel('Fluorescence')

ax2.plot(x2, y2)
ax2.set_title("PSD")
ax2.set_xlabel("Freq (Hz)")
ax2.set_ylabel("Amplitude")

plt.subplots_adjust(hspace=0.3)


In [ ]:
#determine mean amplitude of PSD
power_spectrum_density = np.real(PSD) #takes only real values from the FFT
amplitude = np.sqrt(power_spectrum_density) #need to square root PSD since it is technically fhat^2

mean1s = np.mean(amplitude) #mean amplitude of power from PSD 
print(mean1s)


In [ ]:
#Determine threshold value (can be a right above the noise floor or using a quartile approach), I chose 150% above mean 

#Set all FFT coefficients below the threshold to 0 and perform a reverse FFT to reconstruct signal without excluded frequencies

indices = PSD > mean1s*1.5 #threshold PSD to 150% times mean, this takes out most of the noise 
PSD_clean = PSD * indices
fhat_filt = indices * fhat 
ffilt = np.fft.ifft(fhat_filt)
complex_ifft_output = np.array(ffilt)
float_array = np.real(complex_ifft_output)

#np.savetxt('ffilt.csv', float_array, delimiter=',') #to save filtered signal 

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8))
x1=freq[L]
x2=t
y1=PSD[L]
y2=PSD_clean[L]
y3=ffilt        #can input float_array as well but output seems the same so I'm not sure what the difference is 

ax1.plot(x1, y1)
ax1.set_title("PSD")
ax1.set_xlabel('Freq(Hz)')

ax2.plot(x1, y2)
ax2.set_title("PSD_clean")
ax2.set_xlabel("Freq (Hz)")
ax2.set_ylabel("Magnitude")

ax3.plot(x2, y3)
ax3.set_title('Signal_after_FFT')
ax3.set_xlabel('Time(s)')
ax3.set_ylabel('Intensity')


plt.subplots_adjust(hspace=0.5)

In [ ]:
#Rolling average after FFT with Numpy

#make window size dependent on dt, aka images per second 
from sympy import symbols, Eq, solve 

y = symbols('y')
x = dt 
eq = Eq(4/x, y) #window size is based on time constant of 4 seconds
window_size = solve(eq, y)
window = window_size[0]
window_size = round(window) #need to round window size to nearest whole number 
window_size = int(window_size) #need to convert float to integer 
print(f"The window size for rolling average is {window_size}")
#rolling_mean = np.convolve(float_array, np.ones(window_size)/window_size, mode='valid')    #simpler rolling mean 

from numpy.lib.stride_tricks import sliding_window_view
windows = sliding_window_view(float_array, window_shape=window_size)    #can use np.convolve with mode='same' if you want the output to have the same length as the input (by calculating the edges with partial windows)
rolling_mean = windows.mean(axis=1)

print(f"{len(rolling_mean)} frames in the signal averaged by rolling window")

endpoint = time_end-(window_size-1.00000)*dt 
print(f"Endpoint is {endpoint}s") 

time = np.arange(0, endpoint, dt) #the last window-1 frames are cut off so we have to adjust the time accordingly
print(f"{len(time)} time points")

plt.plot (time, rolling_mean)
plt.title('Signal after FFT + Rolling')
plt.xlabel("Time (s)")
plt.ylabel("Fluorescence")


In [ ]:
#Use Numpy to create percentage-based baseline 
F0 = np.nanmedian(rolling_mean) #can use median because there's not a lot of ongoing activity (since there's not a lot of spikes that would pull the median above true baseline), otherwise would need to identify a specific percentile 
print(F0)

dF_over_F = (rolling_mean - F0) / F0*100 #turned into percentage 

plt.plot(time, dF_over_F)
plt.xlabel("Time (s)")
plt.ylabel("% DF/F")
plt.title("Signal after FFT, Rolling, and DF/F")
plt.ylim(0, 100)

In [ ]:
thresholded = dF_over_F

for i in range(len(thresholded)):
    value = thresholded[i]

    if value < 20: #threshold at 20% dF/F because noise seems to be just below it 
        thresholded[i] = value * 0  #values below threshold turns to 0, although this shortens signal so could find a better way to do this 

plt.plot(time, thresholded)
plt.title("True activity")
plt.ylabel(f"% dF/F")
plt.xlabel("Time(s)")

In [ ]:
from scipy.integrate import simpson
from scipy.signal import peak_widths

t = np.arange(0, endpoint, dt)
auc = simpson(thresholded, t)
print(auc)

plt.plot(t, thresholded, label='%ΔF/F₀')
plt.fill_between(t, thresholded, alpha=0.3, label='Area under curve') #alpha used for blending (values from 0-1) FIX LATER

plt.xlabel('Time (s)')
plt.ylabel('%ΔF/F₀')
plt.title('Calcium Trace with Area Under Curve')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
'''
this part needs to be fixed 
also need to find a way to take out events shorter than 1.5s
'''

from scipy.signal import find_peaks, peak_widths

#fix this so that one signal doesn't have multiple if it's jagged
peaks, _ = find_peaks(thresholded) 
num_of_peaks = len(peaks)
print(f"This signal has {num_of_peaks} peaks at points {peaks}")

peak_w = peak_widths(thresholded, peaks, rel_height=1) #rel_height=1 means it takes the width at the base of the peak 
widths_seconds = peak_w[0] * dt
print(f"These events are {widths_seconds} seconds long")

#freq_events = num_of_peaks/time_end
freq_events = 1/time_end
print(f"The frequency of events in this ROI is {freq_events}/s") 

#average dF/F comes from auc/spike time
activity = auc/time_end
print(f"Total activity is {activity} %dF/F")

plt.figure
plt.plot(t, thresholded, label='%ΔF/F₀')
plt.plot(t[peaks], thresholded[peaks], "x", label='Peaks')
plt.hlines(peak_w[1], t[peak_w[2].astype(int)], t[peak_w[3].astype(int)],
           color="red", label='Widths')

plt.xlabel("Time (s)")
plt.ylabel("%ΔF/F₀")
plt.title("Peak Widths")
plt.legend()
plt.tight_layout()
plt.show()